# Torsion Lab Analysis Code ヽ(^o^)丿

## Define Dimension and Material Properties

In [ ]:
from math import pi

#Specimen Dimensions
L = ##Input the bar length from the lab manual##  # Length of Bar [m]
R = ##Input the bar radius from the lab manual##  # Radius of the Bar [m]
J = ##Calculate the polar 2nd moment of area## #Polar 2nd moment of area [m^4]

#Tensile Properties
E = ##Input the Young's modulus from the tension lab##  # Young's Modulus [Pa]
nu = ##Input the Poisson's ratio from the tension lab## # Poisson's ratio
s_y = ##Input the yield strength from the tension lab## # Yield Strength [Pa]

#Shear Properties
G_th = E/2/(1+nu)  # Theoretical Shear Modulus [Pa]
tau_y = s_y/(3**0.5)   # Yield Shear Stress [Pa]

#Exponential Hardening Properties
H = ##Input the H from the lab manual or your own fit to the tension data##  # Hardening coefficient [Pa]
n = ##Input the n from the lab manual or your own fit to the tension data## # Hardening exponent

## Plot experiment results of Torque vs Angle

### The data is stored in excel sheets
### Here we will extract it

In [ ]:
import pandas as pd #remember that this is the data processing library

#File with all the data
dataFile = 'Torsion data Aut2020.xlsx'

#Import the Steel Data
Steel1 = pd.read_excel(dataFile,header = 3,sheet_name = 'Data 1',usecols = [0,1])
Steel2 = pd.read_excel(dataFile,header = 3,sheet_name = 'Data 2',usecols = [0,1])

#Import the Aluminum Data
Aluminum1 = pd.read_excel(dataFile,header = 3,sheet_name = 'Data 1',usecols = [3,4])
Aluminum2 = pd.read_excel(dataFile,header = 3,sheet_name = 'Data 2',usecols = [3,4])

#Fix some weird stuff with the aluminum import
Aluminum1.columns = Steel1.columns #Rename the columns 
Aluminum1 = Aluminum1.dropna() #Drop NaN values
Aluminum2.columns = Steel2.columns #Rename the columns
Aluminum2 = Aluminum2.dropna() #Drop NaN values

### Now we can process it
### We will show the process for one of the steel data sets

In [ ]:
from math import pi

#Convert the angles to radians
Steel1['Angle (rad)'] = Steel1['Angle (deg)']*pi/180 #(rad)

#Convert the measured force to torque
SteelTorque1 = ##Calculate the torque based on a moment balance for the loading setup##  #(N.m)

### Plot the Results

In [ ]:
import matplotlib.pyplot as plt #Matlab-esque plotting library

#Plot the experimental torque vs twist angle
plt.figure(figsize = (12,8))
plt.plot(Steel1['Angle (rad)'],Steel1['Torque (Nm)'],'.-',label = 'A36 Steel 1')
plt.xlabel('Twist Angle (rad) ',fontsize = 16)
plt.ylabel('Torque (Nm)',fontsize = 16)
plt.title('Torque vs Twist Angle', fontsize = 20)
plt.legend();

### Here are some useful functions to calculate the shear modulus from the data

In [ ]:
from scipy.stats import linregress #This is a linear regression function built into the Scipy library. 
#You can call help(linregress) if you'd like to learn more. Or check out https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html

def maxShearStress(torque,R,J):
    tauMax = ##equation for the max shear stress based on the torque in the elastic region##
    return tauMax

def maxShearStrain(twist,R,L):
    gammaMax = ##equation for the max shear strain based on the twist in the elastic region##
    return gammaMax

def shearModulusFit(twist,torque,R,L,J,a,b):
    '''This is a linear fit to data between the data indices for a and b. Note, this will
    return an error if a or b are outside the length of Strain and Stress.'''
    
    #Find shear stress and shear strain
    shearStrain = [maxShearStrain(x,R,L) for x in twist[a:b]]
    shearStress = [maxShearStress(x,R,J) for x in torque[a:b]]
    
    #Fit the modulus
    G,C,r,P,Err = linregress(shearStrain,shearStress) #The data outputs the slope (G), intercept (C), regression (r) value, P-value and standard error
    #Note: Python lets you save multivariable outputs with a comma, i.e. a,b=[1,2] will give a=1 and b=2
    
    #Make a line for the fit data
    Y = [0.0, max(shearStress)] #this is a list of length 2 for plotting the fit data later
    X = [(y-C)/G for y in Y] #these are points that you can plot to visualize the data being fit, inverted from y=G*x+C, x=(y-C)/G
    return G,C,r,X,Y

### Find the shear modulus from the torsion experiment
### Then compare it with the theoretical shear modulus from the stiffness obtained in the tension lab

In [ ]:
import matplotlib.pyplot as plt #Matlab-esque plotting library

#Fit to the loading data
a = 0
b = 12
G,C,r,X,Y = shearModulusFit(Steel1['Angle (rad)'].values,Steel1['Torque (Nm)'].values,R,L,J,a,b)

#Find the max shear stress and strain from elastic analysis
shearStrain = [maxShearStrain(x,R,L) for x in Steel1['Angle (rad)'][a:b]]
shearStress = [maxShearStress(x,R,J) for x in Steel1['Torque (Nm)'][a:b]]

#Plot the max shear stress and strain and fit
plt.figure(figsize = (12,8))
plt.plot(shearStrain,shearStress,'.-',label = 'A36 Steel 1')
plt.plot(X,Y,'--',label = 'Modulus Fit G='+str(round(G*1e-9,2))+'GPa')
plt.xlabel('Max Shear Strain',fontsize = 16)
plt.ylabel('Max Shear Stress (Pa)',fontsize = 16)
plt.legend();

### Find the Yield Radius for every applied twist
### Remember that this is a theoretical calculation using the material properties obtained in the tension lab
### The big goal here is to compare theoretical plasticity predictions to experimental results

In [ ]:
#Calculate the yield radius
Steel1['Theoretical Yield Radius (m)'] = ##equation for yield radius, hint: use Data['Angle (rad)'] in the calculation##
print(tau_y,L,G_th)
#Plot it to visualize
plt.figure(figsize = (12,8))
plt.plot(Steel1['Angle (rad)'],Steel1['Theoretical Yield Radius (m)'],label = 'Yield Radius')
plt.plot(Steel1['Angle (rad)'],[R]*len(Steel1),label='Bar Radius')
plt.ylim(-0.0002,0.003)
plt.xlabel('Twist Angle(radian) ',fontsize = 16)
plt.ylabel('Radius',fontsize = 16)
plt.title('Yield Radius vs Twist Angle', fontsize = 20)
plt.legend();

## Calculate the theoretical torque from the analytical plasticity model
### Reminder, the bar is theoretically still elastic when the yield radius is larger than the bar radius
### The bar becomes plastic when the yield radius is smaller than the bar radius

In [ ]:
from math import pi

# Finish these functions for the elastic and plastic components of torque
def elasticTorque(r_y,tau_y,G,L,R,theta):
    if r_y > R:
        T_Elastic = ##Include the equation for the elastic torque when the bar is fully elastic##
    else:
        T_Elastic = ##Include the equation for the elastic torque when the bar is plastically deforming##
    return T_Elastic

def plasticTorque(r_y,H,n,L,R,theta):
    if r_y > R:
        T_Plastic =  ##Include the equation for the plastic torque when the bar is fully elastic##
    else:
        T_Plastic = ##Include the equation for the plastic torque when the bar is plastically deforming##
    return T_Plastic

def elasticPlasticTorque(r_y,tau_y,H,n,G,L,R,theta):
    T = elasticTorque(r_y,tau_y,G,L,R,theta) + plasticTorque(r_y,H,n,L,R,theta)
    return T

### Now let's calculate the theoretical torque for everything

In [ ]:
#Initialize some empty variables
eTorque = []
pTorque = []
theoreticalTorque = []

for i in range(len(Steel1)):
    #Create dummy variables to make the code look cleaner
    r_y = Steel1['Theoretical Yield Radius (m)'].values[i]
    theta = Steel1['Angle (rad)'].values[i]
    
    #Run the functions we defined above
    eTorque += [elasticTorque(r_y,tau_y,G_th,L,R,theta)] #The += here is adding values to the list, e.g. a=[1]; a+=[2] -> a=[1,2]
    pTorque += [plasticTorque(r_y,H,n,L,R,theta)]
    theoreticalTorque += [elasticPlasticTorque(r_y,tau_y,H,n,G_th,L,R,theta)]

#And include the values in a Pandas dataframe
Steel1['Theoretical Elastic Torque (Nm)'] = eTorque
Steel1['Theoretical Plastic Torque (Nm)'] = pTorque
Steel1['Theoretical Torque (Nm)'] = theoreticalTorque

### As a brief aside, lets see what we've calculated for the material so far

In [ ]:
#Calling the Pandas dataframe will output the results
Steel1

## Now we can plot the theoretical vs experimental torque!
### We'll also make a plot to see the difference between the theoretical elastic and plastic torque
*Notes:*
- The data for the steel will likely give you strange results if you use the values provided in the manual, please look for an update on Canvas with updated properties.
- These fits are theoretical predictions using an exponential plasticity model. Part of what we're trying to see is how well we can use that theory to fit to real data.

In [ ]:
#Call the plot to make a new figure for the experimental vs theoretical torque v twist
plt.figure(figsize = (12,8))
plt.plot(Steel1['Angle (rad)'],Steel1['Torque (Nm)'],'.-',label = 'Experiment')
plt.plot(Steel1['Angle (rad)'],Steel1['Theoretical Torque (Nm)'],label = 'Theory')
plt.xlabel('Twist Angle(rad) ',fontsize = 16)
plt.ylabel('Torque(Nm)',fontsize = 16)
plt.title('Theoretical vs Experimental Torque',fontsize=20)
plt.legend();

#Call a new plot to make a figure for the theoretical elastic vs plastic contribution to torque
plt.figure(figsize = (12,8))
plt.plot(Steel1['Angle (rad)'],Steel1['Theoretical Elastic Torque (Nm)'],label = 'Elastic Torque')
plt.plot(Steel1['Angle (rad)'],Steel1['Theoretical Plastic Torque (Nm)'],label = 'Plastic Torque')
plt.xlabel('Twist Angle(rad) ',fontsize = 16)
plt.ylabel('Torque(Nm)',fontsize = 16)
plt.title('Elastic vs Plastic Torque Components',fontsize=20)
plt.xlim([0,10]); #We'll zoom in on the twist here to get a closer look at the elastic component
plt.legend();


## Sensitivity Analysis of Hardening Coefficients
### Now we want to see how some variability in the coefficients affects the theoretical fit

In [ ]:
#Try increasing or decreasing the values of H and n to see what effect they have
H = H*1.05; n = n*0.9  # for example

#Then we rerun the theoretical torque calculation from before
#Initialize some empty variables
theoreticalTorque2 = []

for i in range(len(Steel1)):
    #Create dummy variables to make the code look cleaner
    r_y = Steel1['Theoretical Yield Radius (m)'].values[i]
    theta = Steel1['Angle (rad)'].values[i]
    
    #Run the functions we defined above
    theoreticalTorque2 += [elasticPlasticTorque(r_y,tau_y,H,n,G,L,R,theta)]

#And include the values in a Pandas dataframe
Steel1['Theoretical Torque 2 (Nm)'] = theoreticalTorque

### You can then adjust H and n and **plot the results together** with clear label of how you are changing the coefficient

## Hardening Fit
### If you want to do a fit to the hardening data, take the tension data and look at log fitting the plastic yield
#### We will not be giving you code here because this is extra credit
#### Here's a quick rundown of the process:
- Plot the loglog of the stress-strain data for the aluminum or steel
- Plot a marker for the yield point and the ultimate stress
- Determine a suitable "linear" region in that data to fit a hardening slope to
- Take a linear fit of the log of the stress and strain for that section of the data (s = H.e^n, log(s)=log(H)+n.log(e))
- Plot that fit in loglog space to be sure it's adequately capturing the "linear" region of the data